In [71]:
import numpy as np

# 定义映射函数
def map_func(char):
    if char == ".":
        return 0
    elif char.isdigit():
        return int(char)
    elif char == "A":
        return 0
    else:
        raise ValueError(f"Invalid character {char}")

# 打开文件并读取每一行，并对每一行进行映射转换
with open("../map/4.txt", "r") as f:
    result = np.array([list(map(map_func, line.strip())) for line in f])

print(result)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [72]:
x, y = result.nonzero()
print(x,y)
print(result[x,y])

[ 1 29 29 29 29 29 38 38 42 42 42 42 42 46 46 47 47 52 52 55 55 55 55 55
 55 69 69 69 69 69 69 69 75 76 82 82 82 82 82 82 82 82 89 89 95 95 95 95
 95 95] [49  8 24 42 58 84 81 87  8 16 24 42 58 82 86 78 90 42 58  8 24 47 53 76
 92  8 16 24 42 50 58 84 42 84 10 14 18 22 42 49 55 84 42 84 12 20 42 50
 58 84]
[9 1 2 3 1 2 5 4 4 5 5 4 5 4 5 6 6 6 6 6 6 4 5 5 5 6 3 6 5 2 4 1 5 5 4 6 5
 6 5 6 4 5 6 6 6 6 1 2 3 6]


In [73]:
xy = np.stack((x,y),-1)
XY = xy[:, None] - xy[None, :]
XY = np.linalg.norm(XY,2,-1)
print(XY)

[[  0.          49.64876635  37.53664876 ...  94.005319    94.42986816
  100.30453629]
 [ 49.64876635   0.          16.         ...  78.23042886  82.80096618
  100.65783626]
 [ 37.53664876  16.           0.         ...  70.9365914   74.24284477
   89.19641248]
 ...
 [ 94.005319    78.23042886  70.9365914  ...   0.           8.
   34.        ]
 [ 94.42986816  82.80096618  74.24284477 ...   8.           0.
   26.        ]
 [100.30453629 100.65783626  89.19641248 ...  34.          26.
    0.        ]]


In [74]:
dependency_dict = {
    9: [7, 6, 5, 4],
    8: [7],
    7: [6, 5, 4],
    6: [2, 3],
    5: [1, 3],
    4: [1, 2],
    3: [],
    2: [],
    1: []
}
profit = {
    1:3000,
    2:3200,
    3:3400,
    4:7100,
    5:7800,
    6:8300,
    7:29000,
    8:1,
    9:1
}
wb_types = result[x,y]
print(wb_types)
for i, dists in enumerate(XY):
    i_wb_type = wb_types[i]
    for j,d in enumerate(dists):
        j_wb_type = wb_types[j]
        if j_wb_type not in dependency_dict[i_wb_type]:
            XY[i,j] = 99
        else:
            XY[i,j] /= profit[i_wb_type]
print(XY) #费效比矩阵

[9 1 2 3 1 2 5 4 4 5 5 4 5 4 5 6 6 6 6 6 6 4 5 5 5 6 3 6 5 2 4 1 5 5 4 6 5
 6 5 6 4 5 6 6 6 6 1 2 3 6]
[[9.90000000e+01 9.90000000e+01 9.90000000e+01 ... 9.90000000e+01
  9.90000000e+01 1.00304536e+02]
 [9.90000000e+01 9.90000000e+01 9.90000000e+01 ... 9.90000000e+01
  9.90000000e+01 9.90000000e+01]
 [9.90000000e+01 9.90000000e+01 9.90000000e+01 ... 9.90000000e+01
  9.90000000e+01 9.90000000e+01]
 ...
 [9.90000000e+01 9.90000000e+01 9.90000000e+01 ... 9.90000000e+01
  9.90000000e+01 9.90000000e+01]
 [9.90000000e+01 9.90000000e+01 9.90000000e+01 ... 9.90000000e+01
  9.90000000e+01 9.90000000e+01]
 [9.90000000e+01 9.90000000e+01 1.07465557e-02 ... 4.09638554e-03
  3.13253012e-03 9.90000000e+01]]


In [91]:
import itertools
list(itertools.chain(*[(7, 6, 5, 4, 3, 2, 1),(5,),(8,)]))

[7, 6, 5, 4, 3, 2, 1, 5, 8]

In [5]:
def calculate_weights(adjacency_matrix):
    """
    计算每个节点的权重
    """
    return np.sum(adjacency_matrix, axis=1)

def assign_nodes_to_subgraphs(weights, node_types, adjacency_matrix):
    """
    将节点分配到四个子图中
    """
    # 初始化四个子图
    subgraphs = [[] for _ in range(4)]
    subgraph_weights = [0 for _ in range(4)]
    subgraph_node_types = [set() for _ in range(4)]
    
    # 将每个节点分配到最小权重的子图中
    for node_idx in np.argsort(weights):
        node_type = node_types[node_idx]
        # 计算每个子图与当前节点的连接权重和
        subgraph_weights_with_node = []
        for subgraph_idx in range(4):
            subgraph_weight_with_node = subgraph_weights[subgraph_idx] + np.sum(adjacency_matrix[node_idx][subgraphs[subgraph_idx]])
            subgraph_weights_with_node.append(subgraph_weight_with_node)
        # 选择权重和最小的子图并将当前节点加入该子图中
        min_weight_subgraph_idx = np.argmin(subgraph_weights_with_node)
        subgraphs[min_weight_subgraph_idx].append(node_idx)
        subgraph_weights[min_weight_subgraph_idx] = subgraph_weights_with_node[min_weight_subgraph_idx]
        subgraph_node_types[min_weight_subgraph_idx].add(node_type)
        
    return subgraphs, subgraph_node_types

def find_subgraphs(adjacency_matrix, node_types):
    """
    将图分割为四个子图，每个子图包含至少一种类型的节点，并且子图的权重和尽量小
    """
    weights = calculate_weights(adjacency_matrix)
    
    # 重复直到每个子图都包含至少一种类型的节点
    while True:
        subgraphs, subgraph_node_types = assign_nodes_to_subgraphs(weights, node_types, adjacency_matrix)
        if all(len(node_types_in_subgraph) > 0 for node_types_in_subgraph in subgraph_node_types):
            break
        
    return subgraphs


In [4]:
node_type = result[x,y]
node_type

array([1, 2, 4, 2, 3, 5, 3, 1, 6, 2, 5, 4, 3, 1, 7, 8, 7, 1, 2, 5, 4, 3,
       6, 1, 2, 4, 2, 3, 5, 3, 1])

In [11]:
subgraphs = find_subgraphs(XY, node_type)
for i, subgraph in enumerate(subgraphs):
    print(f"Subgraph {i}:")
    print(subgraph)
    print(set(node_type[subgraph]))

Subgraph 0:
[15, 14, 10, 9, 27, 28, 29, 30]
{1, 2, 3, 5, 7, 8}
Subgraph 1:
[17, 22, 20, 21, 26, 25, 24, 23]
{1, 2, 3, 4, 6}
Subgraph 2:
[13, 8, 11, 12, 3, 5, 6, 7]
{1, 2, 3, 4, 5, 6}
Subgraph 3:
[16, 19, 18, 4, 2, 1, 0]
{1, 2, 3, 4, 5, 7}


In [8]:
import numpy as np
from sklearn.cluster import KMeans

def spectral_clustering(adj_matrix, node_types):
    # Step 1: Construct Laplacian matrix
    D = np.diag(np.sum(adj_matrix, axis=1))
    L = D - adj_matrix

    # Step 2: Compute k smallest eigenvectors of L
    k = 4
    eigvals, eigvecs = np.linalg.eigh(L)
    idx = np.argsort(eigvals)[:k]
    X = eigvecs[:, idx]

    # Step 3: Normalize rows of X
    X_norm = np.linalg.norm(X, axis=1, keepdims=True)
    X_norm[X_norm == 0] = 1  # Avoid division by zero
    X_normalized = X / X_norm

    # Step 4: Apply K-Means clustering
    kmeans = KMeans(n_clusters=4).fit(X_normalized)

    # Step 5: Assign nodes to clusters
    clusters = [[] for _ in range(4)]
    for i, label in enumerate(kmeans.labels_):
        clusters[label].append(i)

    # Step 6: Ensure each cluster contains at least one node type
    for i in range(4):
        has_type = False
        for node in clusters[i]:
            if node_types[node] == 1:
                has_type = True
                break
        if not has_type:
            # Find nearest cluster with node type and move closest node
            nearest_cluster = None
            min_distance = np.inf
            for j in range(4):
                if j != i:
                    for node in clusters[j]:
                        if node_types[node] == 1:
                            distance = np.linalg.norm(X_normalized[node] - kmeans.cluster_centers_[i])
                            if distance < min_distance:
                                min_distance = distance
                                nearest_cluster = j
            node_to_move = min(clusters[nearest_cluster], key=lambda x: np.linalg.norm(X_normalized[x] - kmeans.cluster_centers_[i]))
            clusters[nearest_cluster].remove(node_to_move)
            clusters[i].append(node_to_move)

    # Compute weights of each cluster
    weights = [np.sum(adj_matrix[cluster][:, cluster]) for cluster in clusters]

    return clusters, weights


In [9]:
clusters, weights = spectral_clustering(XY, node_type)

In [10]:
print("Clusters:")
for i, cluster in enumerate(clusters):
    print(f"Cluster {i+1}: {cluster}")
print("Weights:", weights)

Clusters:
Cluster 1: [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
Cluster 2: [14, 16, 8]
Cluster 3: [15, 30]
Cluster 4: [17]
Weights: [39922.283628188336, 190.59862173851607, 135.7645019878171, 0.0]


In [12]:
# 找到树的顶端节点
top_node = np.argmax(node_type == 7)

# 初始化已访问节点和未访问节点的列表
visited = [top_node]
unvisited = list(range(len(node_type)))
unvisited.remove(top_node)

# 初始化树的边列表
edges = []

# 遍历所有节点，构建树
while unvisited:
    min_weight = np.inf
    from_node = None
    to_node = None
    # 找到距离已访问节点最近的未访问节点，并构建边
    for i in visited:
        for j in unvisited:
            if XY[i, j] < min_weight:
                min_weight = XY[i, j]
                from_node = i
                to_node = j
    edges.append((from_node, to_node, min_weight))
    visited.append(to_node)
    unvisited.remove(to_node)

# 打印树的边列表
print("Edges of the tree:")
for edge in edges:
    print(edge)

Edges of the tree:
(14, 15, 15.0)
(15, 13, 13.0)
(15, 17, 13.0)
(15, 16, 15.0)
(13, 8, 16.0)
(17, 22, 16.0)
(8, 3, 18.681541692269406)
(3, 4, 11.0)
(22, 26, 19.6468827043885)
(26, 27, 11.0)
(14, 10, 19.697715603592208)
(10, 9, 8.0)
(14, 19, 19.697715603592208)
(19, 18, 8.0)
(16, 11, 20.12461179749811)
(11, 12, 8.0)
(16, 20, 20.12461179749811)
(20, 21, 8.0)
(9, 2, 30.083217912982647)
(2, 1, 4.0)
(1, 0, 4.0)
(12, 5, 30.083217912982647)
(5, 6, 4.0)
(6, 7, 4.0)
(18, 25, 31.04834939252005)
(25, 24, 4.0)
(24, 23, 4.0)
(21, 28, 31.04834939252005)
(28, 29, 4.0)
(29, 30, 4.0)


In [83]:
dependency_dict = {
    9: [(7, 6, 5, 4, 3, 2, 1),],
    8: [7],
    7: [6, 5, 4],
    6: [2, 3],
    5: [1, 3],
    4: [1, 2],
    3: [],
    2: [],
    1: []
}
class TaskNode:
    def __init__(self, type_id:int, workbench=None) -> None:
        self.type_id = type_id     # 这个任务节点的类型id
        self.workbench = workbench # 实际对应的工作台id
        self.children = []   # 子节点列表
        self.make_children()
    
    def make_children(self):
        dep_nodes = dependency_dict[self.type_id]
        children = []
        for d in dep_nodes:
            # 这里不管任务之间是tuple(或关系) 还是int(与关系)，都能同一处理
            # 选出所有可用的工作台
            node_index = np.argwhere(np.isin(wb_types,np.array(d))).squeeze()
            # 查询最高效的
            min_wb = node_index[XY[self.workbench, node_index].argmin()]
            # 因为不知道d是tuple还是int，因此通过wb_types反查类型id
            child = TaskNode(wb_types[min_wb], min_wb)
            child.make_children()
            children.append(child)
        self.children = children
        return self.children

In [84]:
top_node = TaskNode(9, workbench=0)

In [85]:
def print_child(task_node):
    print(task_node.type_id, task_node.workbench)
    if len(task_node.children) == 0:
        return
    print(f"children of {task_node.type_id}:")
    for c in task_node.children:
        print_child(c)
    print("end")

In [86]:
print_child(top_node)

9 0
children of 9:
4 11
children of 4:
1 4
2 2
end
end


array([False, False,  True, False, False,  True, False, False,  True,
       False,  True,  True, False, False, False, False, False, False,
       False,  True,  True, False,  True, False, False,  True, False,
       False,  True, False, False])